In [20]:
#from nltk.tokenize import RegexpTokenizer
from gensim import corpora, models
from gensim.models import Phrases
#from sklearn import decomposition
#import pyLDAvis.gensim as gensimvis
#import pyLDAvis


# initialize tokenizer and stopwords
en_stop = ['a', 'about', 'above', 'after', 'again', 'against', 'all', 'am', 'an', 'and', 'any', 'are', "aren't", 'as',
           'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', "can't", 'cannot',
           'could', "couldn't", 'did', "didn't", 'do', 'does', "doesn't", 'doing', "don't", 'down', 'during', 'each',
           'few', 'for', 'from', 'further', 'had', "hadn't", 'has', "hasn't", 'have', "haven't", 'having', 'he', "he'd",
           "he'll", "he's", 'her', 'here', "here's", 'hers', 'herself', 'him', 'himself', 'his', 'how', "how's", 'i', "i'd",
           "i'll", "i'm", "i've", 'if', 'in', 'into', 'is', "isn't", 'it', "it's", 'its', 'itself', "let's", 'me', 'more',
           'most', "mustn't", 'my', 'myself', 'no', 'nor', 'not', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'ought',
           'our', 'ours', 'ourselves', 'out', 'over', 'own', 'same', "shan't", 'she', "she'd", "she'll", "she's", 'should',
           "shouldn't", 'so', 'some', 'such', 'than', 'that', "that's", 'the', 'their', 'theirs', 'them', 'themselves', 'then',
           'there', "there's", 'these', 'they', "they'd", "they'll", "they're", "they've", 'this', 'those', 'through', 'to',
           'too', 'under', 'until', 'up', 'very', 'was', "wasn't", 'we', "we'd", "we'll", "we're", "we've", 'were', "weren't",
           'what', "what's", 'when', "when's", 'where', "where's", 'which', 'while', 'who', "who's", 'whom', 'why', "why's",
           'with', "won't", 'would', "wouldn't", 'you', "you'd", "you'll", "you're", "you've", 'your', 'yours', 'yourself',
           'yourselves', 'apos', '&apos;m', '&apos;re', '&apos;s', 's', 'I', 'will', 'go', 'get', '(', ')', '?', ':', ';', ',', '.', '!',
           '/', '"', "'", "...","``", "&apos", "&apos;s", "&apos;&apos;", "-lsb-", "-rsb-", "-lcb-", "-rcb-", "-lrb-", "-rrb-",
            "--", " "]
en_stop_stories = ['(', ')', '?', ':', ';', ',', '.', '!', '/', '"', "'", "...","``", "-lsb-", "-rsb-", "-lcb-", "-rcb-", "-lrb-", "-rrb-", "--", " "]

stop_chars = ['<', '>', '.']

# get all lemmas between a <story>-</story>-pair:
stories = []
lemma_stories = []
stop_chars = ['<', '>']

for line in open('1-2-3_story_test.txt'):
    l = line.rstrip()
    #print(l)
    if l == "<story>":
        story = []
        #print(story)
    elif l == "</story>":
        stories.append(story)
    elif not any(stop_char in l for stop_char in stop_chars):
        story.append(l)
#print(stories)

In [21]:
with open('1-2-3_lemma_test.txt') as infile:
    for line in infile:
        l = line.rstrip()
        if l == "<story>":
            story = []
        elif l == "</story>":
            lemma_stories.append(story)
        elif not any(stop_char in l for stop_char in stop_chars):
            if l not in en_stop:
                story.append(l)


In [37]:
print(len(stories))
print(len(lemma_stories))

2881
2432


In [28]:
# It works. But there is a weird behavior here. Have to check it later.
for x in stories:
    if set(x).issubset(en_stop)== True:
        #print(x)
        y = stories.index(x)
        #print(y)
        del stories[y]
    elif len(x) < 20:
        #print(x)
        z = stories.index(x)
        #print(z)
        del stories[z]

#del stories2[1043]
print(len(stories))

2881


In [35]:
# It works. But there is a weird behavior here. Have to check it later.
for x in lemma_stories:
    if set(x).issubset(en_stop)== True:
        #print(x)
        y = lemma_stories.index(x)
        #print(y)
        del lemma_stories[y]
    elif len(x) < 20:
        #print(x)
        z = lemma_stories.index(x)
        #print(z)
        del lemma_stories[z]

#del stories2[1043]
print(len(lemma_stories))

2432


In [38]:

# create dictionary and wordcounts corpus:
dictionary = corpora.Dictionary(lemma_stories)
#print(dictionary.token2id)
#dictionary.save("wordcounts.dict")

print(len(dictionary))

28789


In [39]:
# Bag-of-words representation of the stories.
corpus = [dictionary.doc2bow(story) for story in lemma_stories]
#print(corpus)
#corpora.MmCorpus.serialize("corpus.mm", corpus)
print(len(corpus))
print(len(stories))
print(len(lemma_stories))

# create tf.idf model:
tfidf_model = models.TfidfModel(corpus)
#tfidf_model.save("tfidf_model")
tfidf_corpus = tfidf_model[corpus]
#print(tfidf_corpus)
#print(tfidf_corpus.chunksize)
# corpora.MmCorpus.serialize("tfidf_corpus.mm", tfidf_corpus)
#
# # create topic models:
# LDA
num_topics = 100

lda_model = models.ldamodel.LdaModel(corpus=tfidf_corpus, id2word=dictionary, num_topics=num_topics, update_every=0, chunksize=5000, passes=20)
#lda_model.save("lda_model")
lda_corpus = lda_model[tfidf_corpus]
#corpora.MmCorpus.serialize("lda_corpus.mm", lda_corpus)



2432
2881
2432

Topics by Latent Dirichlet Allocation model
Topic #1 (69, '0.002*"NFL" + 0.002*"Sousa" + 0.001*"Year.i" + 0.001*"deadlyn" + 0.001*"35,000" + 0.001*"TD" + 0.001*"plays" + 0.001*"U." + 0.001*"concussion" + 0.001*"Mora"')


NameError: name 're' is not defined

In [40]:
import re
print("\nTopics by Latent Dirichlet Allocation model")
topics_found_lda = lda_model.print_topics(num_topics=5, num_words=10)
all_topics = lda_model.print_topics(num_topics=100, num_words=10)

# print(topics_found_lda)


counter = 1

for t in topics_found_lda:
    print("Topic #{} {}".format(counter, t))
    words = re.findall('"([^"]+)"', t[1])
    words = ' '.join(words)
    print(words)
    counter += 1

topics = []
for t in all_topics:
    words = re.findall('"([^"]+)"', t[1])
    words = ' '.join(words)
    topics.append(words)

print(topics)
topics.extend(("Stories with actual words", "Number of subelements in stories with actual words", "Stories with lemmas",
               "Number of subelements in stories with lemmas"))
len(topics)




Topics by Latent Dirichlet Allocation model
Topic #1 (36, '0.005*"Jenner" + 0.003*"aderibigbe" + 0.002*"CAITLYN" + 0.002*"oncoming" + 0.002*"22" + 0.002*"hummer" + 0.002*"Hummer" + 0.002*"Vancouver" + 0.002*"Olympic" + 0.002*"Wong"')
Jenner aderibigbe CAITLYN oncoming 22 hummer Hummer Vancouver Olympic Wong
Topic #2 (99, '0.002*"r&amp;b" + 0.002*"auld" + 0.002*"VA" + 0.002*"Lang" + 0.002*"Trotter" + 0.002*"superbug" + 0.002*"Nashville" + 0.002*"antibiotic" + 0.002*"Saab" + 0.002*"rapids"')
r&amp;b auld VA Lang Trotter superbug Nashville antibiotic Saab rapids
Topic #3 (31, '0.010*"insulin" + 0.006*"sugar" + 0.005*"®" + 0.005*"Toujeo" + 0.004*"toujeo" + 0.004*"diabetes" + 0.003*"threatening" + 0.002*"blood" + 0.002*"long-acting" + 0.002*"24-hour"')
insulin sugar ® Toujeo toujeo diabetes threatening blood long-acting 24-hour
Topic #4 (54, '0.008*"banquet" + 0.007*"xr" + 0.006*"truecar" + 0.005*"alzheimer" + 0.003*"Namenda" + 0.003*"NAMENDA" + 0.003*"chicken" + 0.003*"pricing" + 0.003*"o

104

In [48]:
import pandas as pd
import numpy as np

lda_df = pd.DataFrame(columns=range(100))

for i in range(len(lemma_stories)):
    doc = lda_corpus[i]
    for top, prob in doc:
        lda_df.set_value(i, top, prob)
print(doc)

with open("1-2-3_Output.txt", "w") as text_file:
    text_file.write("Doc: %s" % (doc))

lda_df.to_csv("1-2-3_df.csv")

[(17, 0.08175007821444405), (21, 0.10310419118601161), (24, 0.019291091159680196), (50, 0.074412100239124424), (52, 0.066858219420056569), (66, 0.23206271904531833), (93, 0.32661644466856754)]


In [49]:
# lemma_stories2 = [x for x in lemma_stories if x]
# stories2 = [x for x in stories if x]

with open("1-2-3_new_lemma.txt", "w") as lemma_file:
    for item in lemma_stories:
        lemma_file.write("%s\n" % item)
with open("1-2-3_new_story.txt", "w") as story_file:
    for item in stories:
        story_file.write("%s\n" % item)
print(len(lemma_stories))
print(len(stories))
print(len(lda_df))


lda_df

2432
2881
2432


,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.0309879,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.21387,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.300099,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0580225,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.010354,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,0.20359,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,0.0204169,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
# # print("stories:")
# for x in stories2:
#     if set(x).issubset(en_stop)== True:
#         #print(x)
#         y = stories2.index(x)
#         #print(y)
#         del stories2[y]
# #     elif len(x) < 3:
# #         print(x)
# #         print(stories2.index(x))

# #del stories2[1043]
# print(len(stories2))

In [50]:
# lda_df[-1] = stories
# lda_df[-2] = lda_df[-1].apply(lambda x: len(x))
lda_df[-3] = lemma_stories
lda_df[-4] = lda_df[-3].apply(lambda x: len(x))
# # Rows are the stories where -1 column contains the actual words that were used in the stories
# # Columns are the topics found in lda(the words are lemmas) 

In [52]:
lda_df



,0,1,2,3,4,5,6,7,8,9,...,92,93,94,95,96,97,98,99,-3,-4
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[faster, gyro, copter, powerful, pizza, rat, h...",65
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[year, first, think, lot, people, predict, lot...",22
2,0.0309879,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[worst, way, start, come, first, day, January,...",424
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.21387,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[worst, advocate, world, peace, Russian, presi...",188
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.300099,NaN,NaN,NaN,"[flaperon, find, reunion, important, confirm, ...",80
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[many, bright, spot, ahead, come, Hollywood, s...",620
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.010354,NaN,NaN,NaN,"[one, slam, headline, harder, Donald, Trump, a...",428
7,NaN,NaN,NaN,NaN,0.20359,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[White, House, president, Obama, work, hard, n...",273
8,NaN,NaN,NaN,0.0204169,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[come, music, take, bank, lunch, money, Lewis,...",543
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[plenty, thing, catch, attention, online, line...",473


In [53]:

lda_df.to_csv('1-2-3_test.csv')

In [51]:
# lda_df.loc[-1] = topics  # adding a row

# lda_df.index = lda_df.index + 1  # shifting index

# lda_df = lda_df.sort_index() # moving up

# lda_df.reset_index(drop=True, inplace=True)
# lda_df.to_csv('1-2-3_test.csv')

ValueError: cannot set a row with mismatched columns